# **Organización de Datos 75.06 - Trabajo Práctico N°2**
S36. Uno tendería a pensar que mientras más personas participan de una película, más cara es. ¿Cuál es la correlación de Spearman entre estos números? Calcule de forma distribuida.

In [ ]:
!pip install pyspark
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession

from google.colab import drive
drive.mount('/content/drive')

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

cast = sqlContext.read.parquet('/content/drive/MyDrive/Colab Notebooks/TP2/parquet/cast.parquet').rdd
crew = sqlContext.read.parquet('/content/drive/MyDrive/Colab Notebooks/TP2/parquet/crew.parquet').rdd
movies = sqlContext.read.parquet('/content/drive/MyDrive/Colab Notebooks/TP2/parquet/movies.parquet').rdd

In [2]:
actoresPorPeli = cast.map(lambda x: (x.id, 1)).reduceByKey(lambda x,y: x+y)
crewPorPeli = crew.map(lambda x: (x.id, 1)).reduceByKey(lambda x,y: x+y)
personalTotalPorPeli = actoresPorPeli.join(crewPorPeli).map(lambda x: (x[0], x[1][0] + x[1][1]))

pelisBug = ['1997-08-20', '2012-09-29' ,'2014-01-01']
moviesTupla = movies.filter(lambda x: x.id not in pelisBug) #Eliminamos peliculas con id bug (fechas en vez de id).
moviesTupla = moviesTupla.map(lambda x: (int(x.id), int(x.budget)))

pelisInversionYPersonal = moviesTupla.join(personalTotalPorPeli) #Armamos tuplas tipo (id_peli (inversion,personal))

inversionRDD = pelisInversionYPersonal.map(lambda x: x[1][0]) #Obtenemos los RDD separados para calcular la correlación de Spearman.
personalRDD = pelisInversionYPersonal.map(lambda x: x[1][1])

In [4]:
inversion = inversionRDD.sortBy(lambda x: x).zipWithIndex().map(lambda x: (x[0], (x[1]+1, 1))) #Sort y le damos un indice a cada inversion (1, 2, ...)
inversion = inversion.reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).map(lambda x: (x[0], x[1][0] / x[1][1], x[1][1])).sortBy(lambda x: x[0]) #Sumamos indices de inversiones repetidas y dividimos por la cantidad de repeticiones.
inversion = inversion.flatMap(lambda x: [x for i in range(x[2])]) #Volvemos a agregar las inversiones al repetidas (se fueron por el ReduceByKey)

ordenInv = inversion.map(lambda x: x[1]).cache() #Obtenemos la 2da columna, son los indices de orden de cada inversion.

personal = personalRDD.sortBy(lambda x: x).zipWithIndex().map(lambda x: (x[0], (x[1]+1, 1))) #Mismo procedimiento con el RDD de personal.
personal = personal.reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).map(lambda x: (x[0], x[1][0] / x[1][1], x[1][1])).sortBy(lambda x: x[0])
personal = personal.flatMap(lambda x: [x for i in range(x[2])])

ordenPers = personal.map(lambda x: x[1]).cache() #Misma separacion en dos RDD.

#Tengo que hacer esto porque el zip no me funciono.
temp1 = ordenInv.zipWithIndex().map(lambda x: (x[1], x[0]))
temp2 = ordenPers.zipWithIndex().map(lambda x: (x[1], x[0]))
d = temp1.join(temp2).sortBy(lambda x: x[0]).map(lambda x: abs(x[1][0] - x[1][1])) #Genero el RDD de distancias.

suma_d_cuadrado = d.map(lambda x: pow(x, 2)).reduce(lambda a,b : a+b) #Suma de los cuadrados de los elems. de d.
maxOrdenInv = ordenInv.reduce(lambda a,b : a if a > b else b) #Maximo orden del RDD de indices de inversion.
maxOrdenPers = ordenPers.reduce(lambda a,b : a if a > b else b) #Maximo orden del RDD de indices de personal.
n = max(maxOrdenInv, maxOrdenPers)

spearmanDistrib = 1 - (6 * suma_d_cuadrado) / (n * (pow(n, 2) - 1)) #Aplico la formula.

print("La correlacion de Spearman es de", spearmanDistrib, "por lo que hay una dependencia entre la cantidad de personal y la inversion de una pelicula.")


La correlacion de Spearman es de 0.7500261087547841 por lo que hay una dependencia entre la cantidad de personal y la inversion de una pelicula.
